In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
company = pd.read_excel('data/reports-2017-2019.xlsx')
company.groupby("clean_conml")["predicted_report_year"].nunique().value_counts()
company_names = pd.Series(company['conml'].unique())

---------

### Using OpenFIGI with API key

In [3]:
API_KEY = "46c3ca9b-db25-4f62-a57b-9d1047f038df"

In [13]:
import requests
import time

# 1) keep conml + loc
unique_companies = (
    company
    .groupby("clean_conml")
    .agg({
        "conml": lambda x: x.mode().iloc[0],
        "loc":   lambda x: x.dropna().mode().iloc[0] if not x.dropna().empty else None
    })
    .reset_index()
)

# 2) loc -> OpenFIGI exchCode 
loc_to_exch = {
    "US":      "US",
    "Germany": "XETR",
    "UK":      "LON",
    "Canada":  "TOR",
}

SEARCH_URL = "https://api.openfigi.com/v3/search"

DELAY = 3   # less than 20 requests per minute

def map_names_via_search(df, delay=DELAY):
    headers = {
        "Content-Type":      "application/json",
        "X-OPENFIGI-APIKEY": API_KEY
    }
    out = []
    for _, row in df.iterrows():
        name = row["conml"]
        loc  = row["loc"]
        payload = {"query": name}
        # 1. add exchCode if loc is available
        exch = loc_to_exch.get(loc)
        if exch:
            payload["exchCode"] = exch
        # common stock only
        payload["securityType"] = "Common Stock" 

        resp = requests.post(SEARCH_URL, headers=headers, json=payload)
        resp.raise_for_status()
        data = resp.json().get("data", [])

        # filter out non equity
        data = [d for d in data if d.get("securityType") == "Common Stock"]

        if data:
            first = data[0]
            out.append({
                "conml":        name,
                "loc":          loc,
                "ticker":       first.get("ticker"),
                "exchange":     first.get("exchCode"),
                "matched_name": first.get("name")
            })
        else:
            out.append({
                "conml":        name,
                "loc":          loc,
                "ticker":       None,
                "exchange":     None,
                "matched_name": None
            })

        time.sleep(delay)

    return pd.DataFrame(out)

# 3) run search api
df_search = map_names_via_search(unique_companies)
unique_mapped = unique_companies.merge(
    df_search[["conml","ticker","exchange","matched_name"]],
    on="conml", how="left"
)

print(unique_mapped.head())
print(f"unmatched companies", unique_mapped["ticker"].isna().sum())


  clean_conml              conml   loc   ticker exchange          matched_name
0        11ag             1&1 AG  None      DRI       BW                1&1 AG
1          3i       3I Group PLC   GBR   IIIGBP       XW          3I GROUP PLC
2     abengoa         Abengoa SA   ESP  ABGPUSD       XO  ABENGOA SA- B SHARES
3         aia  AIA Group Limited   HKG     None     None                  None
4   aircanada         Air Canada   CAN       AC       CJ            AIR CANADA
unmatched companies 46


so 159-46 = 113 companies matched

In [17]:
unique_mapped.to_csv("data/unique_companies_mapped.csv", encoding="utf-8", index=False)

-------------------------

### yfinance

In [ ]:
from urllib.parse import quote_plus
from rapidfuzz import fuzz

def search_yahoo_smart(df, delay=0.5, fuzz_threshold=85):
    headers = {"User-Agent": "Mozilla/5.0"}
    out = []

    # rough filtering by location
    loc_to_exchs = {
        "Germany": ["XETR"],        
        "UK":      ["LON"],     
        "Canada":  ["TOR"],        
        "US":      ["NMS","NYQ"],   
    }

    loc_to_suffix = {
        "Germany": ".DE",
        "UK":      ".L",
        "Canada":  ".TO",
        "US":      "",     
    }

    for _, row in df.iterrows():
        name = row["conml"]
        loc  = row.get("loc", None)

        # 1) fetch
        url    = f"https://query1.finance.yahoo.com/v1/finance/search?q={quote_plus(name)}"
        quotes = requests.get(url, headers=headers).json().get("quotes", [])

        # 2) filter based on exchangeloc
        if loc in loc_to_exchs:
            quotes = [
                q for q in quotes
                if q.get("exchange") in loc_to_exchs[loc]
            ]

        # 3) then suffix
        suffix = loc_to_suffix.get(loc, "")
        if suffix:
            quotes = [
                q for q in quotes
                if q.get("symbol","").upper().endswith(suffix)
            ]

        # 4) exact match
        exact = [
            q for q in quotes
            if q.get("shortname","").lower()==name.lower()
            or q.get("longname","").lower()==name.lower()
        ]
        if exact:
            pick = exact[0]
            score = 100
        else:
            # 5) fuzzy match
            best_score = 0
            pick = None
            for q in quotes:
                cand = q.get("shortname","") or q.get("longname","")
                sc   = fuzz.token_sort_ratio(name.lower(), cand.lower())
                if sc > best_score:
                    best_score = sc
                    pick = q
            score = best_score
            if best_score < fuzz_threshold:
                pick = None

        # 6) collect
        if pick:
            out.append({
                "conml":        name,
                "matched_name": pick.get("shortname") or pick.get("longname"),
                "ticker":       pick.get("symbol"),
                "exchange":     pick.get("exchange"),
                "fuzz_score":   score
            })
        else:
            out.append({
                "conml":        name,
                "matched_name": None,
                "ticker":       None,
                "exchange":     None,
                "fuzz_score":   None
            })

        time.sleep(delay)

    return pd.DataFrame(out)


In [ ]:
df_yahoo = search_yahoo_smart(unique_companies, delay=1, fuzz_threshold=80)

# drop rows without ticker
df_matched = df_yahoo.dropna(subset=["ticker"]).copy()
print(f"we found: {len(df_matched)}")


we found: 121


In [21]:
df_matched.to_excel("data/unique_companies_yahoo.xlsx", index=False)


In [ ]:
# the original dataframe with None in ticker
# df_yahoo.to_excel("data/unique_companies_yahoo_raw.xlsx", index=False)

---------------------

### get metadata for companies:

In [25]:
import yfinance as yf

meta_out = []
for t in df_matched["ticker"].unique():
    tk = yf.Ticker(t)
    info = tk.info or {}
    meta_out.append({
        "ticker":    t,
        "long_name": info.get("longName"),
        "short_name":info.get("shortName"),
        "country":   info.get("country"),
        "sector":    info.get("sector"),
        "industry":  info.get("industry"),
        "website":   info.get("website"),
    })
    time.sleep(0.5)  

df_meta = pd.DataFrame(meta_out)

# combine metadata with matched tickers
df_enriched = df_matched.merge(df_meta, on="ticker", how="left")
print(df_enriched.head())


               conml                     matched_name  ticker exchange  \
0             1&1 AG  1&1 AG                        I   1U1.F      FRA   
1       3I Group PLC                         3i Group   TGOPY      PNK   
2         Abengoa SA                       ABENGOA SA   AGOAF      OEM   
3  AIA Group Limited  AIA Group Ltd.                R  7A20.F      FRA   
4         Air Canada                       AIR CANADA   AC.TO      TOR   

   fuzz_score          long_name                       short_name  \
0       100.0             1&1 AG  1&1 AG                        I   
1       100.0       3i Group plc                         3i Group   
2       100.0      Abengoa, S.A.                       ABENGOA SA   
3       100.0  AIA Group Limited  AIA Group Ltd.                R   
4       100.0         Air Canada                       AIR CANADA   

          country                  sector                    industry  \
0         Germany  Communication Services            Telecom Servic

In [26]:
def get_kpis_by_year(ticker_str):
    try:
        t = yf.Ticker(ticker_str)
        
        income = t.income_stmt
        balance = t.balance_sheet
        cashflow = t.cashflow

        # gets only the years 2020-2023
        target_years = [str(y) for y in range(2017, 2019)]
        columns = [c for c in income.columns.astype(str) if c[:4] in target_years]

        rows = []
        for year in columns:
            income_row = income.get(year)
            balance_row = balance.get(year)
            cashflow_row = cashflow.get(year)

            rows.append({
                "ticker": ticker_str,
                "year": int(year[:4]),
                "revenue": income_row.get("Total Revenue") if income_row is not None else None,
                "net_income": income_row.get("Net Income") if income_row is not None else None,
                "r_and_d": income_row.get("Research Development") if income_row is not None else None,
                "total_debt": balance_row.get("Total Debt") if balance_row is not None else None,
                "cash": balance_row.get("Cash And Cash Equivalents") if balance_row is not None else None,
                "capex": cashflow_row.get("Capital Expenditures") if cashflow_row is not None else None,
                "source_url": "Yahoo"
            })


        return pd.DataFrame(rows)

    except Exception as e:
        print(f"Error retrieving {ticker_str}: {e}")
        return pd.DataFrame()


In [28]:
kpi_dfs = []
for ticker in df_matched["ticker"].tolist():
    df = get_kpis_by_year(ticker)
    kpi_dfs.append(df)
    time.sleep(0.5)  

kpi_full = pd.concat(kpi_dfs, ignore_index=True)

# calculate profit margin
# kpi_full["profit_margin"] = kpi_full["net_income"] / kpi_full["revenue"]
kpi_full.head()

""


In [29]:
kpi_full.isna().sum()

Series([], dtype: float64)

but if we modify the data range of 2017-2019 to 2020-2023 we do get information --> yfinance only provides financial statements for the last 4 years

In [ ]:
aapl = yf.Ticker("AAPL")
print(aapl.financials.columns)


DatetimeIndex(['2024-09-30', '2023-09-30', '2022-09-30', '2021-09-30',
               '2020-09-30'],
              dtype='datetime64[ns]', freq=None)


thinking of using report from the data lake?

-------------

### start with parquet

I am thinking of making a ticker <--> instrument <--> isin, a 1-1 mapping using symbols_table.parquet. But i haven't found a way to do it, some companies may register multiple tickers in different stock exchanges, when we put KPI on, data may inflate.

In [31]:
import pandas as pd

files = {
    "Compustat Metadata":   "compustat_metadata.parquet",
    "JSON Metadata":        "json_metadata.parquet",
    "LSEG More Metadata":   "lseg_more_metadata.parquet",
    "Symbols Table":        "symbols_table.parquet"
}

for label, path in files.items():
    df = pd.read_parquet(path, engine="pyarrow")
    print(f"=== {label} ({df.shape[0]}×{df.shape[1]}) ===")
    print("Columns:", df.columns.tolist())
    display(df.head(5))


=== Compustat Metadata (104680×13) ===
Columns: ['gvkey', 'ISIN', 'SEDOL', 'cik', 'cusip', 'conml', 'GICS_level_1', 'GICS_level_2', 'GICS_level_3', 'loc', 'weburl', 'curcd', 'exchg']


,gvkey,ISIN,SEDOL,cik,cusip,conml,GICS_level_1,GICS_level_2,GICS_level_3,loc,weburl,curcd,exchg
0,315022,[KR7104540000],[B94CCB9],[],[],[CORENTEC CO LTD],Health Care,Health Care Equipment & Services,Health Care Equipment & Supplies,KOR,None,KRW,298.0
1,317960,[KYG887641085],[BNLPLJ0],[],[],[TIAN GE INTERACTIVE HLDGS],Communication Services,Media & Entertainment,Interactive Media & Services,CHN,None,CNY,170.0
2,8085,[],[],[53495],[675734107],"[October Oil Co, OCTOBER OIL CO]",None,None,None,USA,None,USD,13.0
3,309887,[INE122M01019],[B4JYZ27],[],[],[ESSAR SHIPPING LTD],Energy,Energy,"Oil, Gas & Consumable Fuels",IND,None,INR,219.0
4,29399,[],[],[1655099],[112830104],"[Brookfield Real Assets Income Fund Inc, BROOK...",None,None,None,USA,publicsecurities.brookfield.com,USD,11.0


=== JSON Metadata (423129×20) ===
Columns: ['pdf_local_path', 'pdf_local_path_relative', 'pdfurl', 'pdf_source_path', 'searched_company_name', 'search_rank', 'source', 'md5_fingerprint', 'time_downloaded', 'pdf_title', 'pdf_author', 'pdf_subject', 'pdf_creation_time', 'predicted_language', 'predicted_report_type', 'predicted_company_name', 'predicted_report_year', 'time_processed', 'n_tokenised_sentences', 'n_predicted_initiatives']


,pdf_local_path,pdf_local_path_relative,pdfurl,pdf_source_path,searched_company_name,search_rank,source,md5_fingerprint,time_downloaded,pdf_title,pdf_author,pdf_subject,pdf_creation_time,predicted_language,predicted_report_type,predicted_company_name,predicted_report_year,time_processed,n_tokenised_sentences,n_predicted_initiatives
0,/srv/data/mrei/davinci/davinci_data_v11_2022_u...,000009/Tadano_Ltd_integrated_report2021_en_7_g...,https://www.tadano.com/upload/docs/integrated_...,None,Tadano Ltd,1,google_search,77bb2990686e1509f8ba790a6a2eeffe,1699496911.4256902,,None,None,D:20220114140730+09'00',en,sustainability report,Tadano Ltd,2021,1699498549.3246822,735,8
1,/srv/data/mrei/davinci/davinci_data_v11_2022_u...,000009/Tadano_Ltd_integrated_report2022_en_for...,https://www.tadano.com/upload/docs/integrated_...,None,Tadano Ltd,3,google_search,c8095caf9d14ee73c44de89102380bcb,1699496912.334108,None,None,None,D:20221209152016+09'00',en,sustainability report,Tadano Ltd,2022,1699498546.4602056,2341,9
2,/srv/data/mrei/davinci/davinci_data_v11_2022_u...,000009/Taikisha_Ltd_pdf-index-2021-01_7r58ksqc...,https://www.taikisha-group.com/sustainability/...,None,Taikisha Ltd,3,google_search,ad88313421d39a8afa9e32ed03ee3c75,1699488723.331302,None,None,None,D:20211220191732+09'00',en,integrated report,Taikisha Ltd,2021,1699489970.6086187,6464,11
3,/srv/data/mrei/davinci/davinci_data_v11_2022_u...,000009/Taikisha_Ltd_pdf-index-2022-02_nllsl2qx...,https://www.taikisha-group.com/sustainability/...,None,Taikisha Ltd,1,google_search,217c0a48e25415855343f4f1d9316259,1699489131.509729,None,None,None,D:20221210111703+09'00',en,integrated report,Taikisha Ltd,2022,1699489970.9746637,6498,5
4,/srv/data/mrei/davinci/davinci_data_v11_2022_u...,000009/Takasago_International_Corp_Takasago20S...,https://assets.takasago.com/prod/default/s3fs-...,None,Takasago International Corp,1,google_search,78c1b3239aee77f27bd9010e8b52e1f2,1699499833.9023306,,None,None,D:20220920103340+09'00',en,sustainability report,Takasago International Corp,2021,1699500939.8893101,3216,10


=== LSEG More Metadata (818323×14) ===
Columns: ['Instrument', 'TR.MIC', 'TR.ExchangeCode', 'TR.CompanyMarketCap', 'TR.CommonName', 'TR.GICSSector', 'TR.GICSIndustryGroup', 'TR.GICSIndustry', 'TR.BusinessSummary', 'TR.AlsoKnownAsName', 'TR.FormerlyKnownAsName', 'TR.DoingBusinessAsName', 'TR.ImmediateParent', 'TR.UltimateParent']


,Instrument,TR.MIC,TR.ExchangeCode,TR.CompanyMarketCap,TR.CommonName,TR.GICSSector,TR.GICSIndustryGroup,TR.GICSIndustry,TR.BusinessSummary,TR.AlsoKnownAsName,TR.FormerlyKnownAsName,TR.DoingBusinessAsName,TR.ImmediateParent,TR.UltimateParent
0,LP68841490,NaN,LIP,NaN,Desjardins Target 2026 Investment Grade Bond C,None,None,None,None,None,None,None,None,None
1,LP68841491,NaN,LIP,NaN,Desjardins Target 2026 Investment Grade Bond C,None,None,None,None,None,None,None,None,None
2,LP68841495,NaN,LIP,NaN,Desjardins Target 2027 Investment Grade Bond C,None,None,None,None,None,None,None,None,None
3,LP68841496,NaN,LIP,NaN,Desjardins Target 2027 Investment Grade Bond C,None,None,None,None,None,None,None,None,None
4,LP68841497,NaN,LIP,NaN,Desjardins Target 2027 Investment Grade Bond C,None,None,None,None,None,None,None,None,None


=== Symbols Table (641490×12) ===
Columns: ['DocumentTitle', 'RIC', 'IssueISIN', 'SEDOL', 'FundClassLipperID', 'TickerSymbol', 'Instrument', 'CUSIP', 'IssuerOAPermID', 'TR.HQCountryCode', 'TR.InstrumentIsActive', 'TR.IsPrimaryInstrument']


,DocumentTitle,RIC,IssueISIN,SEDOL,FundClassLipperID,TickerSymbol,Instrument,CUSIP,IssuerOAPermID,TR.HQCountryCode,TR.InstrumentIsActive,TR.IsPrimaryInstrument
0,"LUX IM European Equities DXL EUR, Open-End Fun...",LP65032832,LU0255269960,None,65032832,None,LP65032832,None,None,LU,True,False
1,"LUX IM Pictet Asian Equities DXL EUR, Open-End...",LP65032835,LU0255273566,None,65032835,None,LP65032835,None,None,LU,True,False
2,"LUX IM Morgan Stanley US Equities DXL EUR, Ope...",LP65032839,LU0255272089,None,65032839,None,LP65032839,None,None,LU,True,False
3,"LUX IM Pictet Asian Equities DXBL EUR, Open-En...",LP65032840,LU0255236019,None,65032840,None,LP65032840,None,None,LU,True,False
4,"JPM Europe Strategic Value I Dist EUR, Open-En...",LP65032853,LU0248049685,B11Z367,65032853,None,LP65032853,None,None,LU,True,False


In [32]:
# 1. filter for active and primary instruments
dfsymbols = pd.read_parquet(files["Symbols Table"], engine="pyarrow")
df_map = dfsymbols[
    (dfsymbols["TR.InstrumentIsActive"] == True) &
    (dfsymbols["TR.IsPrimaryInstrument"] == True)
]

df_map = (
    df_map[["TickerSymbol","IssueISIN","Instrument"]]
    .drop_duplicates()
    .rename(columns={
        "TickerSymbol": "ticker",
        "IssueISIN":    "isin"
    })
)

df_map.head()

,ticker,isin,Instrument
4434,ZIL2,DE0007856023,ZILGn.DE
4541,ZILL,RU0009086193,ZILL.MM
4542,ZIL,PK0033501013,ZILL.PSX
4781,ZILU,MKZILU101012,ZILU.MKE
4939,ZIM,GB0061284906,ZIM.AX
